## Decision Trees Implementation ##


# 1) Loading Data


In [30]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import pandas as pd


##### Data Setup ######
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
data = load_breast_cancer()
X = data.data           
y = data.target        
feature_names = data.feature_names  
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

# Split the remaining 30% into 15% val and 15% test
# 15% is half of 30% → test_size = 0.5
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

## 2)Node Class


In [31]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_leaf_node(self):
        if self.value is not None:
            return True
        return False

## 3) Tree Class

In [ ]:
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=2, max_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.root=None
        self.max_features=max_features
        self.all_gains=[]
        
    def fit(self, X, y):
        self.all_gains = [0] * X.shape[1]
        self.root = self._grow_tree(X, y)
        
    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))
        if(depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        best_feature, best_threshold = self._best_split(X, y)
        left_mask=X[:, best_feature] <= best_threshold
        right_mask=X[:, best_feature] > best_threshold
        left_data, right_data = X[left_mask], X[right_mask]
        left_labels, right_labels = y[left_mask], y[right_mask]
        left_child=self._grow_tree(left_data, left_labels, depth+1)
        right_child=self._grow_tree(right_data, right_labels, depth+1)
        return Node(feature=best_feature, threshold=best_threshold, left=left_child, right=right_child)
    #### Helper Functions ####
    
    def _best_split(self, X, y):
        n_samples, n_features = X.shape
        best_overall_gain = -1
        best_feature = None
        best_threshold = None

        if self.max_features is None or self.max_features > n_features:
            features_to_consider = range(n_features)
        else:
            features_to_consider = np.random.choice(n_features, self.max_features, replace=False)



        for feature in features_to_consider:
            
            # Compute all possible midpoints for this feature
            values = np.sort(np.unique(X[:, feature]))
            thresholds = [(values[i] + values[i+1]) / 2 for i in range(len(values) - 1)]

            best_feature_gain = -1  # reset best gain for THIS feature

            for threshold in thresholds:
                gain = self._info_gain(y, X[:, feature], threshold)

                # Update best-overall split
                if gain > best_overall_gain:
                    best_overall_gain = gain
                    best_feature = feature
                    best_threshold = threshold

                # Update best gain for THIS feature
                if gain > best_feature_gain:
                    best_feature_gain = gain

            # Save best gain of this feature for feature ranking
            self.all_gains[feature] += best_feature_gain*n_samples

        return best_feature, best_threshold

    def _info_gain(self, y, feature_column, threshold):
        parent_entropy=self.entropy(y)
        left_mask=feature_column <= threshold
        right_mask=feature_column > threshold
        left_y=y[left_mask]
        right_y=y[right_mask]
        if len(y[left_mask]) == 0 or len(y[right_mask]) == 0:
            return 0
        left_entropy=self.entropy(left_y)
        right_entropy=self.entropy(right_y)
        weighted_entropy=(len(left_y)/len(y))*left_entropy + (len(right_y)/len(y))*right_entropy
        info_gain=parent_entropy - weighted_entropy
        return info_gain
    def entropy(self, y):
        class_labels, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        entropy = -np.sum(probabilities * np.log2(probabilities + 1e-9))
        return entropy
    def _most_common_label(self, y):
        class_labels, counts = np.unique(y, return_counts=True)
        most_common = class_labels[np.argmax(counts)]
        return most_common
    def predict(self, X):
        root=self.root
        predictions=[]
        for x in X:
            node = self.root
            while not node.is_leaf_node():
                if x[node.feature] <= node.threshold:
                    node=node.left
                else:
                    node=node.right
            predictions.append(node.value)
        return np.array(predictions)
    def ranked_features(self):
       
        if not self.all_gains:
            return 0
        gains=np.argsort(self.all_gains)[::-1]
        table = pd.DataFrame({
        "Feature": [feature_names[i] for i in gains],
        "Information Gain": [self.all_gains[i] for i in gains]
        })
        print(table)

## 4) Analytics Functions

## 4.1) *Hyperparameter Testing*


In [33]:
######## Analytics Functions ##########
def hyperparameter_tuning(X_train,y_train,X_val,y_val):
    best_acc=0
    best_params = None
    accuracy=[]
    best_tree=None
    max_depth={2, 4, 6, 8, 10} 
    min_samples_split = {2, 5, 10}
   
    results = []

    for depth in max_depth:
        for min_split in min_samples_split:
            tree = DecisionTree(max_depth=depth, min_samples_split=min_split)
            tree.fit(X_train, y_train)
            preds = tree.predict(X_val)
            acc = np.mean(preds == y_val)
            print(f"Depth={depth}, MinSplit={min_split}, ValAcc={acc:.4f}")
            results.append((depth, min_split, acc))
            if acc > best_acc:
                best_acc = acc
                best_params = (depth, min_split)
         
    print("\nBest Parameters:", best_params, "Validation Accuracy:", best_acc)
    return best_params, results
 


In [34]:
best_params, tuning_results = hyperparameter_tuning(X_train, y_train, X_val, y_val)
best_depth, best_min_split = best_params

Depth=2, MinSplit=2, ValAcc=0.9176
Depth=2, MinSplit=10, ValAcc=0.9176
Depth=2, MinSplit=5, ValAcc=0.9176
Depth=4, MinSplit=2, ValAcc=0.9882
Depth=4, MinSplit=10, ValAcc=0.9882
Depth=4, MinSplit=5, ValAcc=0.9882
Depth=6, MinSplit=2, ValAcc=0.9647
Depth=6, MinSplit=10, ValAcc=0.9647
Depth=6, MinSplit=5, ValAcc=0.9647
Depth=8, MinSplit=2, ValAcc=0.9882
Depth=8, MinSplit=10, ValAcc=0.9647
Depth=8, MinSplit=5, ValAcc=0.9647
Depth=10, MinSplit=2, ValAcc=0.9882
Depth=10, MinSplit=10, ValAcc=0.9647
Depth=10, MinSplit=5, ValAcc=0.9647

Best Parameters: (4, 2) Validation Accuracy: 0.9882352941176471


## 4.2) *Changing only Max Depth*


In [35]:
def max_depth_analysis(X_train,y_train,X_val,y_val):
    max_depth={2, 4, 6, 8, 10} 
    min_samples_split=2
    table = []
    for depth in max_depth:
        tree = DecisionTree(max_depth=depth, min_samples_split=min_samples_split)
        tree.fit(X_train, y_train)
        preds = tree.predict(X_val)
        val_acc = np.mean(preds == y_val)
        train_preds = tree.predict(X_train)
        train_acc = np.mean(train_preds == y_train)
        print(f"Depth={depth}, ValAcc={val_acc:.4f}")
        
        table.append((depth, train_acc, val_acc))

    print("\nDepth | Train Acc | Val Acc")
    for row in table:
        print(f"{row[0]:5} | {row[1]:9.4f} | {row[2]:8.4f}")



In [36]:
max_depth_analysis(X_train, y_train, X_val, y_val)

Depth=2, ValAcc=0.9176
Depth=4, ValAcc=0.9882
Depth=6, ValAcc=0.9647
Depth=8, ValAcc=0.9882
Depth=10, ValAcc=0.9882

Depth | Train Acc | Val Acc
    2 |    0.9523 |   0.9176
    4 |    0.9925 |   0.9882
    6 |    0.9950 |   0.9647
    8 |    1.0000 |   0.9882
   10 |    1.0000 |   0.9882


##  4.3) *Tree Complexity and Model Performance*

To evaluate the effect of model complexity, we analyze how varying the **maximum depth** of the tree influences performance.

We examine:
- **Training Accuracy**
- **Validation Accuracy**
- **Overfitting / underfitting behavior**

**What we expect:**
- At **low depth**, the model underfits → high bias, low accuracy.
- As depth increases, training accuracy rises.
- Validation accuracy improves at first, then declines when the tree becomes too complex.
  This indicates **overfitting**.

**How results are presented:**
- A table or plot showing accuracy vs. tree depth.
- The optimal depth is chosen where validation accuracy peaks.
- This depth is then used to retrain on training + validation data and evaluated on the test set.

**Conclusion:**
- Tree depth strongly affects generalization.
- The best performance occurs at the depth that balances bias and variance.


## 4.4) *Accuarcy measures on Test set*

In [37]:
def train_val_combine(X_train, y_train, X_val, y_val):
    X_train_val = np.vstack((X_train, X_val))
    y_train_val = np.hstack((y_train, y_val))
    return X_train_val, y_train_val

def evaluate_performance(y_true, y_pred):
    print("===== PERFORMANCE METRICS =====")
    print(f"Accuracy:  {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred, average=None)}")
    print(f"Recall:    {recall_score(y_true, y_pred, average=None)}")
    print(f"F1-score:  {f1_score(y_true, y_pred, average=None)}\n")

    print("===== CLASSIFICATION REPORT =====")
    print(classification_report(y_true, y_pred, digits=4))
    cm = confusion_matrix(y_true, y_pred)
    print("===== CONFUSION MATRIX =====")
    print(cm)

In [45]:
final_tree = DecisionTree(max_depth=best_depth, min_samples_split=best_min_split)
X_train_val, y_train_val = train_val_combine(X_train, y_train, X_val, y_val)
max_depth_analysis(X_train, y_train, X_val, y_val)
final_tree.fit(X_train_val, y_train_val)
test_preds_trees = final_tree.predict(X_test)
evaluate_performance(y_test, test_preds_trees)

Depth=2, ValAcc=0.9176
Depth=4, ValAcc=0.9882
Depth=6, ValAcc=0.9647
Depth=8, ValAcc=0.9882
Depth=10, ValAcc=0.9882

Depth | Train Acc | Val Acc
    2 |    0.9523 |   0.9176
    4 |    0.9925 |   0.9882
    6 |    0.9950 |   0.9647
    8 |    1.0000 |   0.9882
   10 |    1.0000 |   0.9882
===== PERFORMANCE METRICS =====
Accuracy:  0.8837
Precision: [0.86666667 0.89285714]
Recall:    [0.8125     0.92592593]
F1-score:  [0.83870968 0.90909091]

===== CLASSIFICATION REPORT =====
              precision    recall  f1-score   support

           0     0.8667    0.8125    0.8387        32
           1     0.8929    0.9259    0.9091        54

    accuracy                         0.8837        86
   macro avg     0.8798    0.8692    0.8739        86
weighted avg     0.8831    0.8837    0.8829        86

===== CONFUSION MATRIX =====
[[26  6]
 [ 4 50]]


## 4.5) *Ranking of Features*

In [39]:
final_tree.ranked_features()

                    Feature  Information Gain
0            worst symmetry          0.918296
1      worst concave points          0.918296
2          worst smoothness          0.918296
3         worst compactness          0.918296
4             worst texture          0.918296
5                 mean area          0.918296
6           mean smoothness          0.918296
7               mean radius          0.918296
8              mean texture          0.918296
9          smoothness error          0.459148
10           mean perimeter          0.459148
11            mean symmetry          0.459148
12          perimeter error          0.459148
13  worst fractal dimension          0.316689
14             radius error          0.316689
15           mean concavity          0.316689
16     concave points error          0.316689
17          concavity error          0.316689
18  fractal dimension error          0.316689
19             worst radius          0.316689
20               worst area       

## 4.6) *Overfitting Analysis: Training vs. Validation Performance*

### Overview
Overfitting occurs when a model learns the training data too well, including its noise and peculiarities, resulting in poor generalization to unseen data. For decision trees, this is controlled primarily by **tree depth** and **minimum samples per split**.

### Key Observations

**Training vs. Validation Accuracy Gap:**
- As tree depth increases, training accuracy typically continues to improve (approaching 100%)
- Validation accuracy initially improves but then plateaus or decreases
- The gap between training and validation accuracy indicates overfitting

**Optimal Depth:**
- At shallow depths (depth=2-4), both accuracies are lower → **underfitting** (high bias)
- At moderate depths (depth=6-8), validation accuracy peaks → **balanced model**
- At deep depths (depth=10+), large gap appears → **overfitting** (high variance)

### How to Identify Overfitting

1. **Growing Gap**: If `Training Accuracy - Validation Accuracy > 0.05`, the model is likely overfitting
2. **Plateau Effect**: Validation accuracy stops improving while training accuracy keeps rising
3. **Noise Learning**: Deep trees fit training noise rather than true patterns

### Strategies to Reduce Overfitting

- **Limit max_depth**: Prevent trees from growing too complex
- **Increase min_samples_split**: Require more samples to create splits (reduces tree complexity)
- **Pruning**: Remove branches that don't significantly improve validation accuracy
- **Use validation data**: Select hyperparameters based on validation performance, not training

### Conclusion

The results from `max_depth_analysis()` demonstrate the bias-variance tradeoff. The optimal model depth is where validation accuracy is highest. Using deeper trees may achieve higher training accuracy but sacrifices generalization, indicating overfitting.

# Part D: Random Forest

## Random Forest Implementation

In [40]:
class RandomForest:
    def __init__(self,min_sample_split=2 , max_depth=4 , n_trees=10, n_features=None):
        self.min_sample_split=min_sample_split
        self.max_depth=max_depth
        self.n_trees=n_trees
        self.n_features=n_features
        self.trees=[]
    
    def fit(self,X,y):
        self.trees=[]
        for _ in range(self.n_trees):
            tree= DecisionTree(min_samples_split=self.min_sample_split, max_depth=self.max_depth, max_features=self.n_features)
            X_bt, y_bt=self._bootstrap_sample(X,y)
            tree.fit(X_bt,y_bt)
            self.trees.append(tree)
            
    def _bootstrap_sample(self,X,y):
        n_samples=X.shape[0]
        indices=np.random.choice(n_samples,size=n_samples,replace=True)
        return X[indices], y[indices]
    
    def predict(self,x):
        predictions=np.array([tree.predict(x) for tree in self.trees])
        majority_votes=[np.bincount(pred).argmax() for pred in predictions.T]
        return np.array(majority_votes)

## Hyperparameter Tuning and Evaluation

In [41]:
def random_forest_tuning(X_train, y_train, X_val, y_val, d):
    T_values = [5, 10, 30, 50]
    max_features_values = [int(np.sqrt(d)), int(d/2)]
    best_acc = -1
    best_params = None
    results = []
    print("\n===== Random Forest Validation Results =====")
    print(f"{'T':<10} | {'max_features':<15} | {'Val Accuracy':<10}")
    print("-" * 45)
    for T in T_values:
        for mf in max_features_values:
            rf = RandomForest(
                n_trees=T,
                n_features=mf,
                max_depth=4,            
                min_sample_split=2  
            )		
            rf.fit(X_train, y_train)
            preds = rf.predict(X_val)
            acc = accuracy_score(y_val, preds)

            print(f"{T:<10} | {mf:<15} | {acc:.4f}")

            results.append((T, mf, acc))

            if acc > best_acc:
                best_acc = acc
                best_params = (T, mf)

    print("\nBEST RANDOM FOREST PARAMETERS:")
    print(f"T = {best_params[0]}, max_features = {best_params[1]}")
    print(f"Validation Accuracy = {best_acc:.4f}")

    return best_params, results

## best (T, max_features) combination

In [ ]:
d = X_train.shape[1]   # number of features

# Step 1 — tune
best_params, _ = random_forest_tuning(X_train, y_train, X_val, y_val, d)
best_T, best_mf = best_params


===== Random Forest Validation Results =====
T          | max_features    | Val Accuracy
---------------------------------------------
5          | 5               | 0.9765
5          | 15              | 0.9647
10         | 5               | 0.9647
10         | 15              | 0.9765
30         | 5               | 0.9882
30         | 15              | 0.9882
50         | 5               | 0.9765
50         | 15              | 0.9882

BEST RANDOM FOREST PARAMETERS:
T = 30, max_features = 5
Validation Accuracy = 0.9882


## Retrain the random forest on training + validation

In [44]:
# Step 2 — combine train and val sets
X_train_val, y_train_val = train_val_combine(X_train, y_train, X_val, y_val)

# train final model
final_rf = RandomForest(
    n_trees=best_T,
    n_features=best_mf,
    max_depth=4,
    min_sample_split=2
)
print("\n==== TRAINING FINAL RANDOM FOREST MODEL ====")
final_rf.fit(X_train_val, y_train_val)

# test

test_preds = final_rf.predict(X_test)

print("\n==== FINAL TEST PERFORMANCE ====")
evaluate_performance(y_test, test_preds)


==== TRAINING FINAL RANDOM FOREST MODEL ====

==== FINAL TEST PERFORMANCE ====
===== PERFORMANCE METRICS =====
Accuracy:  0.9070
Precision: [0.92857143 0.89655172]
Recall:    [0.8125     0.96296296]
F1-score:  [0.86666667 0.92857143]

===== CLASSIFICATION REPORT =====
              precision    recall  f1-score   support

           0     0.9286    0.8125    0.8667        32
           1     0.8966    0.9630    0.9286        54

    accuracy                         0.9070        86
   macro avg     0.9126    0.8877    0.8976        86
weighted avg     0.9085    0.9070    0.9055        86

===== CONFUSION MATRIX =====
[[26  6]
 [ 2 52]]


# compare the performance of a single decision tree with the random forest

In [48]:
print("\n==== DECISION TREE TEST PERFORMANCE ====\n")

evaluate_performance(y_test, test_preds_trees)

print("\n\n\n==== FINAL RANDOM FOREST TEST PERFORMANCE ====\n")
evaluate_performance(y_test, test_preds)


==== DECISION TREE TEST PERFORMANCE ====

===== PERFORMANCE METRICS =====
Accuracy:  0.8837
Precision: [0.86666667 0.89285714]
Recall:    [0.8125     0.92592593]
F1-score:  [0.83870968 0.90909091]

===== CLASSIFICATION REPORT =====
              precision    recall  f1-score   support

           0     0.8667    0.8125    0.8387        32
           1     0.8929    0.9259    0.9091        54

    accuracy                         0.8837        86
   macro avg     0.8798    0.8692    0.8739        86
weighted avg     0.8831    0.8837    0.8829        86

===== CONFUSION MATRIX =====
[[26  6]
 [ 4 50]]



==== FINAL RANDOM FOREST TEST PERFORMANCE ====

===== PERFORMANCE METRICS =====
Accuracy:  0.9070
Precision: [0.92857143 0.89655172]
Recall:    [0.8125     0.96296296]
F1-score:  [0.86666667 0.92857143]

===== CLASSIFICATION REPORT =====
              precision    recall  f1-score   support

           0     0.9286    0.8125    0.8667        32
           1     0.8966    0.9630    0.928

Model Performance (Accuracy & Generalization)
### Single Decision Tree

Learns simple to very complex patterns depending on depth.

Can achieve high training accuracy.

Often overfits the training data.

Performance on unseen (test) data is usually unstable.

### Random Forest

Combines many decision trees built on:

Different bootstrap samples (bagging).

Different random subsets of features.

Predictions are made by majority voting (classification) or averaging (regression).

Achieves higher and more stable test accuracy than a single tree.

Much better generalization.

### Conclusion on Performance:
Random Forest almost always outperforms a single decision tree on real-world data.

## discuss the effect on bias and variance.

### Effect on bias

| Model             | Bias Level                               | Explanation                                                                 |
| ----------------- | ---------------------------------------- | --------------------------------------------------------------------------- |
| **Decision Tree** | **Low Bias**                             | It can fit very complex patterns, especially when deep.                     |
| **Random Forest** | **Slightly Higher Bias (but still low)** | Each tree is slightly restricted by randomness, but still flexible overall. |


### Effect on variance

| Model             | Variance Level           | Explanation                                                |
| ----------------- | ------------------------ | ---------------------------------------------------------- |
| **Decision Tree** |  **Very High Variance** | Small data changes can produce completely different trees. |
| **Random Forest** |  **Low Variance**       | Averaging many trees cancels out individual errors.        |


### Bias–Variance Tradeoff Summary

| Model                    | Bias       | Variance | Overfitting | Stability     |
| ------------------------ | ---------- | -------- | ----------- | ------------- |
| **Single Decision Tree** | Low        |  High   |  High      |  Unstable    |
| **Random Forest**        | Low–Medium |  Low    |  Low       |  Very Stable |
